# Playground

This is a space for experimentation.

In [66]:
import random
import itertools
import numpy as np
from datetime import datetime
from tic_tac_toe import WorldTTT
from utility import get_opposite_symbol, track_time

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [62]:
world_ttt = WorldTTT()

In [63]:
world_ttt.actions

[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]

In [70]:
dt = datetime.now()

'034653'

'30032024'

In [68]:
datetime.strptime(datetime.now(), ""

'2024-03-30 03:43:27.575458'